In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, InputLayer, Input, Flatten, Conv1D, MaxPooling1D
import optuna
import openpyxl
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from keras.losses import SparseCategoricalCrossentropy
from keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import ReduceLROnPlateau

E:\Anaconda\envs\tfnew\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
ds = pd.read_csv('intraindata2n.csv', header=None)
ds = ds.sample(frac=1).reset_index(drop=True)
x_train = np.array(ds.iloc[:, 5:])
y1_train = np.array(ds[[0]])
y2_train = np.array(ds[[1]])
y3_train = np.array(ds[[2]])
y4_train = np.array(ds[[3]])
y5_train = np.array(ds[[4]])
ds = pd.read_csv('intestdata2.csv', header=None)
ds = ds.sample(frac=1).reset_index(drop=True)
x_test = np.array(ds.iloc[:, 5:])
y1_test = np.array(ds[[0]])
y2_test = np.array(ds[[1]])
y3_test = np.array(ds[[2]])
y4_test = np.array(ds[[3]])
y5_test = np.array(ds[[4]])
ds = pd.read_csv('invalidation2n.csv', header=None)
ds = ds.sample(frac=1).reset_index(drop=True)
x_val = np.array(ds.iloc[:, 5:])
y1_val = np.array(ds[[0]])
y2_val = np.array(ds[[1]])
y3_val = np.array(ds[[2]])
y4_val = np.array(ds[[3]])
y5_val = np.array(ds[[4]])

In [ ]:
miny3 = np.min(y3_train)
maxy3 = np.max(y3_train)
miny4 = np.min(y4_train)
maxy4 = np.max(y4_train)
miny5 = np.min(y5_train)
maxy5 = np.max(y5_train)
y3_train = (y3_train - miny3) / (maxy3 - miny3)
y4_train = (y4_train - miny4) / (maxy4 - miny4)
y5_train = (y5_train - miny5) / (maxy5 - miny5)

y3_test = (y3_test - miny3) / (maxy3 - miny3)
y4_test = (y4_test - miny4) / (maxy4 - miny4)
y5_test = (y5_test - miny5) / (maxy5 - miny5)

y3_val = (y3_val- miny3) / (maxy3 - miny3)
y4_val = (y4_val - miny4) / (maxy4 - miny4)
y5_val = (y5_val - miny5) / (maxy5 - miny5)


In [ ]:
def objective(trial):
    in1 = Input(shape=x_train[0].shape)

    x = Dense(500, activation='relu')(in1)
    x = Dense(500, activation='relu')(x)
    x = Dense(500, activation='relu')(x)
    x = Dense(500, activation='relu')(x)
    out1 = Dense(2, activation='softmax')(x)

    x = Dense(800, activation='relu')(in1)
    x = Dense(800, activation='relu')(x)
    x = Dense(800, activation='relu')(x)
    x = Dense(800, activation='relu')(x)
    out2 = Dense(3, activation='softmax')(x)

    num_layers_m1=trial.suggest_int('num_layers_m1' , 1,10 )
    num_neurons_m1=trial.suggest_int('num_neurons_m1' ,100, 2000)
    lr=trial.suggest_float('lr', 0.0001, 0.001, log=True)


    x_p1=Dense(num_neurons_m1, activation='relu')(in1)
    for i in range(num_layers_m1):
        x_p1=Dense(num_neurons_m1, activation='relu')(x_p1)
    x_p1=Dense(1, activation='linear')(x_p1)

    x_p2=Dense(num_neurons_m1, activation='relu')(in1)
    for i in range(num_layers_m1):
        x_p2=Dense(num_neurons_m1, activation='relu')(x_p2)
    x_p2=Dense(1, activation='linear')(x_p2)

    x_p3=Dense(num_neurons_m1, activation='relu')(in1)
    for i in range(num_layers_m1):
        x_p3=Dense(num_neurons_m1, activation='relu')(x_p3)
    x_p3=Dense(1, activation='linear')(x_p3)
    model=Model(inputs=in1, outputs=[ out1, out2, x_p1, x_p2, x_p3])

    optimizer=tf.keras.optimizers.Adam(lr)
    model.compile(optimizer=optimizer, loss=['SparseCategoricalCrossentropy', 'SparseCategoricalCrossentropy','mse', 'mse', 'mse'],
              metrics=[['accuracy'], ['accuracy'],['mse'], ['mse'], ['mse']])
    early=EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

    model.fit(x_train,( y1_train, y2_train, y3_train, y4_train, y5_train), validation_data=(x_val,[y1_val, y2_val, y3_val, y4_val, y5_val]), batch_size=128, epochs=500, verbose=0,validation_batch_size=128, shuffle=True, callbacks=[early])

    score=model.evaluate(x_test, [y1_test, y2_test, y3_test, y4_test, y5_test])
    return score[0]

In [ ]:
study= optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial= study.best_trial
print(" Value: {}".format(trial.value))
print("  Params  ")
for key, value in trial.params.items():
    print("  {}: {}".format(key, value))